Q1. Implement simple neural network models using numerical
datasets. Solve a regression problem by building a neural network
with L2 (Mean Squared Error) loss.

Importing all necessary library

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
import torch   # main librar for pytorch
import torch.nn as nn # for creating neural network
import torch.optim as optim # optimization
from sklearn.model_selection import train_test_split # for spliting dataset
from sklearn.preprocessing import StandardScaler #  for feature engineering

Loading dataset

In [35]:
data=pd.read_csv('/content/insurance - insurance.csv')

In [36]:
data

,age,sex,bmi,smoker,region,charges
0,19,female,27.900,yes,southwest,16884.92400
1,18,male,33.770,no,southeast,1725.55230
2,28,male,33.000,no,southeast,4449.46200
3,33,male,22.705,no,northwest,21984.47061
4,32,male,28.880,no,northwest,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,no,northwest,10600.54830
1334,18,female,31.920,no,northeast,2205.98080
1335,18,female,36.850,no,southeast,1629.83350
1336,21,female,25.800,no,southwest,2007.94500


Shape of Dataset

In [48]:
data.shape

(1338, 6)

Info

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         1338 non-null   int64  
 1   bmi         1338 non-null   float64
 2   region      1338 non-null   object 
 3   charges     1338 non-null   float64
 4   sex_male    1338 non-null   bool   
 5   smoker_yes  1338 non-null   bool   
dtypes: bool(2), float64(2), int64(1), object(1)
memory usage: 44.6+ KB


Find categorial data and performing One-Hot Encoding

In [52]:

categorical_cols = data.select_dtypes(include=['object', 'category']).columns
print("\nCategorical Columns Found:", categorical_cols)


if len(categorical_cols) > 0:
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

print("\nDataset after encoding:")
print(data.head())



Categorical Columns Found: Index([], dtype='object')

Dataset after encoding:
   age     bmi      charges  sex_male  smoker_yes  region_northwest  \
0   19  27.900  16884.92400     False        True             False   
1   18  33.770   1725.55230      True       False             False   
2   28  33.000   4449.46200      True       False             False   
3   33  22.705  21984.47061      True       False              True   
4   32  28.880   3866.85520      True       False              True   

   region_southeast  region_southwest  
0             False              True  
1              True             False  
2              True             False  
3             False             False  
4             False             False  


Identify categorical columns

Seperating feature and target

In [54]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values


Spliting dataset & normalization

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Convert Data to PyTorch Tensors

In [57]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


/tmp/ipython-input-4200861718.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
/tmp/ipython-input-4200861718.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test  = torch.tensor(X_test, dtype=torch.float32)
/tmp/ipython-input-4200861718.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
/tmp/ipython-input-4200861718.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach()

Define Neural Network Model

In [59]:
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


Initialize Model, Loss & Optimizer

In [60]:
input_size = X_train.shape[1]

model = RegressionNN(input_size)

criterion = nn.MSELoss()          # L2 loss
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [61]:
epochs = 100

for epoch in range(epochs):
    model.train()

    predictions = model(X_train)
    loss = criterion(predictions, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: 0.1801
Epoch [20/100], Loss: 0.1429
Epoch [30/100], Loss: 0.1246
Epoch [40/100], Loss: 0.1209
Epoch [50/100], Loss: 0.1183
Epoch [60/100], Loss: 0.1167
Epoch [70/100], Loss: 0.1155
Epoch [80/100], Loss: 0.1145
Epoch [90/100], Loss: 0.1136
Epoch [100/100], Loss: 0.1128


In [62]:
model.eval()

with torch.no_grad():
    test_predictions = model(X_test)
    test_loss = criterion(test_predictions, y_test)

print("\nTest MSE Loss:", test_loss.item())



Test MSE Loss: 0.11523900926113129


In [63]:
print("\nActual values:", y_test[:5].view(-1))
print("Predicted values:", test_predictions[:5].view(-1))



Actual values: tensor([0., 0., 0., 0., 0.])
Predicted values: tensor([ 0.5137,  0.0020, -0.0663, -0.0183,  0.0335])
